# Scikit-Learn Reference

---

# Table of Contents

## [Data Class ](#Data-Class)

## [Load Data ](#Load-Data)

## [Prep Data ](#Prep-Data)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Bag of Words Vectorization ](#Bag-of-Words-Vectorization)

## [Classification ](#Classification)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Linear SVM ](#Linear-SVM)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Decision Tree ](#Decision-Tree)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Naive Bayes ](#Naive-Bayes)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Logistic-Regression ](#Logistic-Regression)

## [Evaluation ](#Evaluation)

## [Tuning our Model (With Grid Search) ](#Tuning-our-Model-(With-Grid-Search))

## [Saving Model ](#Saving-Model)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Save Model ](#Save-Model)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Load Model ](#Load-Model)

---

# Data Class

In [1]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # score of 4 or 5
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
        
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

---

# Load Data

In [2]:
import json

file_name = "./data/sentiment/books_small_10000.json"
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review["reviewText"], review["overall"]))
reviews[342].sentiment

'POSITIVE'

---

# Prep Data

In [3]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [4]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


### Bag of Words Vectorization

In [5]:
 from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    
# This book is great !
# This book was so bad

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0])

  (0, 3355)	0.1260987450189101
  (0, 2686)	0.09706609761484508
  (0, 8879)	0.05616628705856641
  (0, 3968)	0.06528702197527632
  (0, 8817)	0.10916307526324438
  (0, 8665)	0.07226492943548314
  (0, 3781)	0.090702954029304
  (0, 4782)	0.07300505853450096
  (0, 2946)	0.13721357596418915
  (0, 5802)	0.1195156804693861
  (0, 5549)	0.0680698344934807
  (0, 1891)	0.1630424442326433
  (0, 3908)	0.1329482398057699
  (0, 2198)	0.15734499445248573
  (0, 8711)	0.1630424442326433
  (0, 5512)	0.055876688497626684
  (0, 1187)	0.18074033972744635
  (0, 5005)	0.13964709895768268
  (0, 285)	0.08298491747007937
  (0, 2863)	0.15268983902650163
  (0, 1971)	0.14534454873784025
  (0, 2424)	0.1630424442326433
  (0, 7303)	0.0749550642981604
  (0, 423)	0.0322900864028065
  (0, 8462)	0.14875396119485296
  :	:
  (0, 8760)	0.09559625255652344
  (0, 371)	0.10994875774823414
  (0, 8704)	0.06901559691390033
  (0, 6904)	0.11335817020524684
  (0, 3761)	0.11159659825673793
  (0, 4406)	0.18074033972744635
  (0, 7790)	0.1

---

# Classification

### Linear SVM

In [6]:
from sklearn import svm
clf_svm = svm.SVC(kernel="linear")
clf_svm.fit(train_x_vectors, train_y)
print(clf_svm.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
In this book we learn that no matter our circumstances we can overcome and rise above them, and we can be loved! Sometimes it takes hard things and maybe even tragic things to happen in our lives to help us see more clearly and to grow. I  enjoyed this book in the series. Heather Tullis is a great writer and makes the story interesting with some twists and turns. I look forward to reading the rest of the series. 
 POSITIVE


### Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

print(clf_dec.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['NEGATIVE']
In this book we learn that no matter our circumstances we can overcome and rise above them, and we can be loved! Sometimes it takes hard things and maybe even tragic things to happen in our lives to help us see more clearly and to grow. I  enjoyed this book in the series. Heather Tullis is a great writer and makes the story interesting with some twists and turns. I look forward to reading the rest of the series. 
 POSITIVE


### Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

print(clf_gnb.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
In this book we learn that no matter our circumstances we can overcome and rise above them, and we can be loved! Sometimes it takes hard things and maybe even tragic things to happen in our lives to help us see more clearly and to grow. I  enjoyed this book in the series. Heather Tullis is a great writer and makes the story interesting with some twists and turns. I look forward to reading the rest of the series. 
 POSITIVE


### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

clf_log = DecisionTreeClassifier()
clf_log.fit(train_x_vectors, train_y)

print(clf_log.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['NEGATIVE']
In this book we learn that no matter our circumstances we can overcome and rise above them, and we can be loved! Sometimes it takes hard things and maybe even tragic things to happen in our lives to help us see more clearly and to grow. I  enjoyed this book in the series. Heather Tullis is a great writer and makes the story interesting with some twists and turns. I look forward to reading the rest of the series. 
 POSITIVE


---

# Evaluation

In [10]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6418269230769231
0.6442307692307693
0.6802884615384616


In [11]:
# F1 Score
from sklearn.metrics import f1_score
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.80582524 0.80952381]
[0.63569682 0.64775414]
[0.64251208 0.64593301]
[0.66997519 0.68997669]


In [12]:
train_y.count(Sentiment.POSITIVE)

436

---

# Tuning our Model (With Grid Search)

In [22]:
from sklearn.model_selection import GridSearchCV

parameters = {"kernel": ("linear", "rbf"), "C": (1, 4, 8, 16, 32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [21]:
print(clf.best_params_)
print(clf.best_score_)


{'C': 4, 'kernel': 'rbf'}
0.8290968801313628


---

# Saving Model

### Save Model

In [26]:
import pickle

with open("./models/sentiment_classifier.pkl", "wb") as f:
    pickle.dump(clf, f)

### Load Model

In [27]:
with open("./models/sentiment_classifier.pkl", "rb") as f:
    loaded_clf = pickle.load(f)

In [28]:
loaded_clf.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')